In [3]:
import os
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from utils import show_image

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
tf.__version__

'2.4.0-rc1'

In [ ]:
%matplotlib inline

Move raw collected data into the correct hierarchy

* Example <br>
  train
    ---> class1
        ---> class_image_here
    ---> class2
        ---> class_image_here
        .
        .
        .

In [35]:
train = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test = ImageDataGenerator( rescale = 1.0/255. )

In [36]:
train_gen = train.flow_from_directory(
    "../images/train/", color_mode="rgb", batch_size = 20, class_mode="categorical", target_size = (150, 150)
)
test_gen = test.flow_from_directory(
    "../images/test/", color_mode="rgb", batch_size = 20, class_mode="categorical", target_size = (150, 150)
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Load Pre-Trained Model, and freeze the weights.

In [ ]:
base_model = InceptionV3(
    input_shape = (160, 160, 3), 
    include_top = False, 
    weights = 'imagenet'
)

for layer in base_model.layers:
    layer.trainable = False
base_model.trainable = False

Define the last layer of the CNN, a fully connected layer.

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
#base_model.summary()
#model.summary()

Train the pre-trained model's last layer

In [ ]:
model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)